In [ ]:

import numpy as np
import matplotlib.pyplot as plt
# If you need to import from a local utils.py, uncomment and adjust the following lines:
import sys
import os
import jax.numpy as jnp
# Construct the full path to the folder
folder_path = r'C:\Users\Petrb\Desktop\DTU\3rdSemester\02477_BAYESIAN_MACHINE_LEARNING'

# Add the folder to the Python path
sys.path.append(folder_path)

# Now you can import the utils module
from EXAMS.EXAM_2024.Task_2.utils import *

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from kernel_gaussian_classes import * 

: 

In [3]:
x = jnp.array([-2.00, 0.00, 2.00])[:, None]
y = jnp.array([-2.01, 1.41, 0.23])[:, None]


kappa = jnp.sqrt(2)
ell = 2.0
sigma = 0.5 

In [20]:
kernel = StationaryIsotropicKernel(kappa=kappa, lengthscale=ell, kernel_fun=squared_exponential)

K_prior = kernel.contruct_kernel(x, x)

print(K_prior)

[[1.9999999  1.2130612  0.27067053]
 [1.2130612  1.9999999  1.2130612 ]
 [0.27067053 1.2130612  1.9999999 ]]


In [30]:
kernel = StationaryIsotropicKernel(kappa=kappa, lengthscale=ell, kernel_fun=squared_exponential)

model = GaussianProcessRegression(x, y, kernel=kernel, sigma=sigma, kappa=kappa, lengthscale=ell)

In [35]:
mu, var = model.predict_f(x)
print(mu)
print(var)

[[-1.5283369 ]
 [ 0.8865025 ]
 [ 0.42873892]]
[[ 0.20844185  0.02778709 -0.00998175]
 [ 0.02778697  0.19226015  0.02778709]
 [-0.00998175  0.02778709  0.20844197]]


In [43]:
def rbf(X1, X2, length_scale=1.0, kappa=1.0, debug=False):
    """
    Computes a “squared exponential style” kernel (here using an L₁ based form) plus a linear term.
    
    Parameters
    ----------
    X1 : np.ndarray, shape (n1, d)
        First batch of points (n1 points in d dimensions).
    X2 : np.ndarray, shape (n2, d)
        Second batch of points (n2 points in d dimensions).
    length_scale : float
        Lengthscale parameter ell > 0 (default 1.0).
    variance : float
        Variance (amplitude) parameter σ² (default 1.0).  Currently not used in your return statement,
        but you could multiply the whole kernel by this if desired.
    
    Returns
    -------
    K : np.ndarray, shape (n1, n2)
        Kernel matrix where
          K[i,j] = 1 * (1 + (‖X1[i] − X2[j]‖₁ / (2 ℓ²)))⁻¹  +  X1[i] · X2[j]
    """
    # ensure length_scale is positive and nonzero
    l = np.abs(length_scale) + 1e-12
    

    
    # 2) Compute pairwise differences via broadcasting:
    #    diff[i,j,k] = X1[i,k] - X2[j,k]
    #    shape of diff: (n1, n2, d)
    diff = X1[:, None, :] - X2[None, :, :]
    
    # 3) Sum absolute differences over the last axis to get L1 distance:
    #    sqdist[i,j] = sum_k |diff[i,j,k]|
    #    shape of sqdist: (n1, n2)*
    sqdist = np.linalg.norm((diff)**2, axis=2)
    
    
    # 4) Compute the kernel:
    #    A) “Squared‐exponential–style” term (but using L1 distance here):
    #         (1 + sqdist / (2 ℓ²))⁻¹
    #    B) Plus a linear term X1·X2ᵀ
    #    Final shape: (n1, n2)
    
    
    if debug:
        print("*" * 50)
        print("Debugging information:")
        
        print("Length scale (l):", l)
        # 1) Print shapes for debugging
        #   X1 shape: (n1, d)
        print("X1 shape before:", X1.shape)
        #   After adding a new axis: (n1, 1, d)
        print("X1[:, None, :] shape:", X1[:, None, :].shape)
        #   X2 shape: (n2, d)
        print("X2 shape before:", X2.shape)
        #   After adding a new axis: (1, n2, d)
        print("X2[None, :, :] shape:", X2[None, :, :].shape)
        #  diff shape: (n1, n2, d)
        print("diff shape:", diff.shape)
        #  sqdist shape: (n1, n2)
        print("sqdist shape:", sqdist.shape)
        #  K shape: (n1, n2)
        print("K shape:", np.exp(-0.5 * sqdist ) + 2 .shape)
        # print K 
        print("K:", np.exp(-0.5 * sqdist ) + 2 )
        print("*" * 50)
        print("*" * 50)
    
    return np.exp(-0.5 * sqdist ) + 2 

In [46]:
xstar = np.array([[2.0]])

C = rbf(xstar, xstar)

print("prior mean m(x*): 0.0 (assumed)")
print("prior variance C  :\n", C)

prior mean m(x*): 0.0 (assumed)
prior variance C  :
 [[3.]]
